In [1]:
import pandas as pd
import numpy as np
from initializer import init, CSV_BASE_DIR

init()

# Pandas Combining

## Appending new rows to DataFrame

### Append without using `append()` (using `loc`)

In [6]:
df = pd.DataFrame(columns=['a', 'b'])

#### Add Data as `list`

In [7]:
df.loc[0] = [1, 2]
df.loc['Foo'] = [1, 2]
df.head()

,a,b
0,1,2
Foo,1,2


#### Add data as `dict`

In [8]:
df.loc[len(df)] = {'a': 'Foo', 'b': 'Bar'}
df.head()

,a,b
0,1,2
Foo,1,2
2,Foo,Bar


#### Add data as `Series`

In [9]:
df.loc['Bar'] = pd.Series({'a': 'Hello', 'b': 'World'})
df.head()

,a,b
0,1,2
Foo,1,2
2,Foo,Bar
Bar,Hello,World


In [11]:
# 덮어쓰기 가능
df.loc['Bar'] = pd.Series({'a': 'Pandas', 'b': 'Hater'})
df

,a,b
0,1,2
Foo,1,2
2,Foo,Bar
Bar,Pandas,Hater


> 위 방식들은 모두 `inplace` 방식이라고 한다.

`inplace` 방식은 기존 데이터를 직접 접근해서 값을 변경하는 것을 의미한다.
다른 방식들은 원본 DataFrame을 보존하고 복사된 새로운 DataFrame을 사용한다.

### Append using `append()` ... ~~Deprecated~~

- 위의 `loc`과는 다르게 not in-place(returns a new copy of the DataFrame)
- `append()`: it only accept
    - `DataFrame`
    - `Series`
    - `Dictionary`
    - list of these(Not `list` itself)

In [12]:
names_df = pd.DataFrame(
    {
        'name': ['철수', '영희', '영수', '영미'],
        'age': [12, 13, 14, 15]
    },
    index=['Canada', 'Canada', 'USA', 'USA']
)

names_df

,name,age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15


In [14]:
# ERROR! index를 어떤 값을 설정할지 모르기 때문
# > TypeError: Can only append a dict if ignore_index=True
names_df.append({'name': '명수', 'age': 10})

/var/folders/19/zmm26ll95zb7drffrjsdx8_h0000gn/T/ipykernel_52780/4199018086.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df.append({'name': '명수', 'age': 10})


TypeError: Can only append a dict if ignore_index=True

#### `ignore_index=True`

- 이전 index 값들이 reset 된다.

In [32]:
names_df.append(
    {'name': '명수', 'age': 10},
    ignore_index=True
)
pd.concat([names_df, pd.DataFrame([['명수', 10]], columns=['name', 'age'])], ignore_index=True)

/var/folders/19/zmm26ll95zb7drffrjsdx8_h0000gn/T/ipykernel_52780/2352780293.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df.append(


,name,age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,10


,name,age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,10


In [33]:
# list 가능
names_df.append(
    [{'name': '명수', 'age': 10}, {'name': '명호', 'age': 11}],
    ignore_index=True
)
new_df = pd.DataFrame([['명수', 10], ['명호', 11]], columns=['name', 'age'])
pd.concat([names_df, new_df], ignore_index=True)

/var/folders/19/zmm26ll95zb7drffrjsdx8_h0000gn/T/ipykernel_52780/2589138142.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df.append(


,name,age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,10
5,명호,11


,name,age
0,철수,12
1,영희,13
2,영수,14
3,영미,15
4,명수,10
5,명호,11


#### 기존 index 유지는 `append` 시 dict 대신 `Series`를 전달

In [36]:
new_series = pd.Series({'name': '명수', 'age': 10}, name='Korea')
names_df.append(new_series)

new_df = pd.DataFrame([new_series], columns=['name', 'age'])
pd.concat([names_df, new_df])

/var/folders/19/zmm26ll95zb7drffrjsdx8_h0000gn/T/ipykernel_52780/2572880235.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  names_df.append(new_series)


,name,age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15
Korea,명수,10


,name,age
Canada,철수,12
Canada,영희,13
USA,영수,14
USA,영미,15
Korea,명수,10


## `concat`, `join`, and `merge`

### `concat()`

- DataFrame or Series object를 vertically or horizontally 연결
- index(or columns)에 대해 align(not values - join과 merge와의 차이점)
- Default to outer join
    - operation axis에 따라 concat 되는 object column or index가 union

In [37]:
import FinanceDataReader as fdr

In [38]:
samsung_df = fdr.DataReader('005930', '2009-01-01', '2017-12-31')
kodex_df = fdr.DataReader('069500', '2016-01-01', '2017-12-31')

samsung_df.head()
kodex_df.head()

pd.concat([samsung_df, kodex_df]).head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,9070,9400,8850,9400,460533,0.042
2009-01-05,9600,9740,9470,9520,609528,0.013
2009-01-06,9640,10040,9590,9960,901449,0.046
2009-01-07,10100,10480,9980,10480,827163,0.052
2009-01-08,10359,10380,10060,10160,613566,-0.031


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2016-01-04,21135,21143,20774,20786,7949903,-0.020
2016-01-05,20733,20980,20733,20916,14251222,0.006
2016-01-06,20905,20927,20647,20764,12478969,-0.007
2016-01-07,20709,20786,20506,20565,12867202,-0.010
2016-01-08,20437,20719,20392,20711,14064157,0.007


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,9070,9400,8850,9400,460533,0.042
2009-01-05,9600,9740,9470,9520,609528,0.013
2009-01-06,9640,10040,9590,9960,901449,0.046
2009-01-07,10100,10480,9980,10480,827163,0.052
2009-01-08,10359,10380,10060,10160,613566,-0.031


In [41]:
pd.concat([samsung_df, kodex_df[['Open', 'High']]]).head(2)
pd.concat([samsung_df, kodex_df[['Open', 'High']]]).tail(2)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2009-01-02,9070,9400,8850.000,9400.000,460533.000,0.042
2009-01-05,9600,9740,9470.000,9520.000,609528.000,0.013


,Open,High,Low,Close,Volume,Change
Date,,,,,,
2017-12-27,29090,29330,NaN,NaN,NaN,NaN
2017-12-28,29344,29730,NaN,NaN,NaN,NaN


In [43]:
pd.concat(
    [samsung_df, kodex_df],
    keys=['삼성전자', 'KODEX200'],
    names=['ticker']
)

pd.concat([samsung_df, kodex_df], keys=['삼성전자', 'KODEX200'], names=['종목명', '날짜'])

Open   High    Low  Close    Volume  Change
ticker   Date                                                    
삼성전자     2009-01-02   9070   9400   8850   9400    460533   0.042
         2009-01-05   9600   9740   9470   9520    609528   0.013
         2009-01-06   9640  10040   9590   9960    901449   0.046
         2009-01-07  10100  10480   9980  10480    827163   0.052
         2009-01-08  10359  10380  10060  10160    613566  -0.031
...                    ...    ...    ...    ...       ...     ...
KODEX200 2017-12-21  29471  29525  28968  28976   9449472  -0.019
         2017-12-22  29049  29175  28976  29115   9360619   0.005
         2017-12-26  29172  29348  28990  28997   8390432  -0.004
         2017-12-27  29090  29330  29030  29338  13964805   0.012
         2017-12-28  29344  29730  29339  29705   8343692   0.013

[2718 rows x 6 columns]

Open   High    Low  Close    Volume  Change
종목명      날짜                                                      
삼성전자     2009-01-02   9070   9400   8850   9400    460533   0.042
         2009-01-05   9600   9740   9470   9520    609528   0.013
         2009-01-06   9640  10040   9590   9960    901449   0.046
         2009-01-07  10100  10480   9980  10480    827163   0.052
         2009-01-08  10359  10380  10060  10160    613566  -0.031
...                    ...    ...    ...    ...       ...     ...
KODEX200 2017-12-21  29471  29525  28968  28976   9449472  -0.019
         2017-12-22  29049  29175  28976  29115   9360619   0.005
         2017-12-26  29172  29348  28990  28997   8390432  -0.004
         2017-12-27  29090  29330  29030  29338  13964805   0.012
         2017-12-28  29344  29730  29339  29705   8343692   0.013

[2718 rows x 6 columns]

In [44]:
pd.concat([samsung_df, kodex_df], axis=1)

,Open,High,Low,Close,Volume,Change,Open,High,Low,Close,Volume,Change
Date,,,,,,,,,,,,
2009-01-02,9070,9400,8850,9400,460533,0.042,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,9600,9740,9470,9520,609528,0.013,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,9640,10040,9590,9960,901449,0.046,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,10100,10480,9980,10480,827163,0.052,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,10359,10380,10060,10160,613566,-0.031,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,51000,51060,49100,49140,312486,-0.034,29471.000,29525.000,28968.000,28976.000,9449472.000,-0.019
2017-12-22,49400,49960,49240,49700,223993,0.011,29049.000,29175.000,28976.000,29115.000,9360619.000,0.005
2017-12-26,49760,50100,48200,48200,320797,-0.030,29172.000,29348.000,28990.000,28997.000,8390432.000,-0.004


In [45]:
pd.concat([samsung_df, kodex_df], axis=1, keys=['삼성전자', 'KODEX200'])

삼성전자                                      KODEX200            \
             Open   High    Low  Close  Volume Change      Open      High   
Date                                                                        
2009-01-02   9070   9400   8850   9400  460533  0.042       NaN       NaN   
2009-01-05   9600   9740   9470   9520  609528  0.013       NaN       NaN   
2009-01-06   9640  10040   9590   9960  901449  0.046       NaN       NaN   
2009-01-07  10100  10480   9980  10480  827163  0.052       NaN       NaN   
2009-01-08  10359  10380  10060  10160  613566 -0.031       NaN       NaN   
...           ...    ...    ...    ...     ...    ...       ...       ...   
2017-12-21  51000  51060  49100  49140  312486 -0.034 29471.000 29525.000   
2017-12-22  49400  49960  49240  49700  223993  0.011 29049.000 29175.000   
2017-12-26  49760  50100  48200  48200  320797 -0.030 29172.000 29348.000   
2017-12-27  48960  49560  48460  49360  214872  0.024 29090.000 29330.000   
2017-12-28  49560  50960  49500  50960  179709  0.032 29344.000 29730.000   

                                                    
                 Low     Close       Volume Change  
Date                                                
2009-01-02       NaN       NaN          NaN    NaN  
2009-01-05       NaN       NaN          NaN    NaN  
2009-01-06       NaN       NaN          NaN    NaN  
2009-01-07       NaN       NaN          NaN    NaN  
2009-01-08       NaN       NaN          NaN    NaN  
...              ...       ...          ...    ...  
2017-12-21 28968.000 28976.000  9449472.000 -0.019  
2017-12-22 28976.000 29115.000  9360619.000  0.005  
2017-12-26 28990.000 28997.000  8390432.000 -0.004  
2017-12-27 29030.000 29338.000 13964805.000  0.012  
2017-12-28 29339.000 29705.000  8343692.000  0.013  

[2229 rows x 12 columns]

In [49]:
# join - How to handle indexes on other axis(es)
# concat의 대상이 되는 axis 말고, 다른 axis의 index에 대해 어떻게 join 할 것인가
df_1 = pd.DataFrame(
    [
        ['A0', 'B0', 'C0', 'D0'],
        ['A1', 'B1', 'C1', 'D1'],
        ['A2', 'B2', 'C2', 'D2'],
        ['A3', 'B3', 'C3', 'D3'],
    ],
    index=[0, 1, 2, 3],
    columns=['A', 'B', 'C', 'D']
)
df_2 = pd.DataFrame(
    [
        ['B2', 'D2', 'F2'],
        ['B3', 'D3', 'F3'],
        ['B6', 'D6', 'F6'],
        ['B7', 'D7', 'F7'],
    ],
    index=[2, 3, 6, 7],
    columns=['B', 'D', 'F']
)
pd.concat([df_1, df_2], axis=1, join='inner')

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


In [50]:
pd.concat([samsung_df, kodex_df], axis=1, keys=['삼성전자', 'KODEX200'], join='inner')

삼성전자                                     KODEX200                \
             Open   High    Low  Close  Volume Change     Open   High    Low   
Date                                                                           
2016-01-04  25200  25200  24100  24100  306939 -0.044    21135  21143  20774   
2016-01-05  24040  24360  23720  24160  216002  0.002    20733  20980  20733   
2016-01-06  24160  24160  23360  23500  366752 -0.027    20905  20927  20647   
2016-01-07  23320  23660  23020  23260  282388 -0.010    20709  20786  20506   
2016-01-08  23260  23720  23260  23420  257763  0.007    20437  20719  20392   
...           ...    ...    ...    ...     ...    ...      ...    ...    ...   
2017-12-21  51000  51060  49100  49140  312486 -0.034    29471  29525  28968   
2017-12-22  49400  49960  49240  49700  223993  0.011    29049  29175  28976   
2017-12-26  49760  50100  48200  48200  320797 -0.030    29172  29348  28990   
2017-12-27  48960  49560  48460  49360  214872  0.024    29090  29330  29030   
2017-12-28  49560  50960  49500  50960  179709  0.032    29344  29730  29339   

                                    
            Close    Volume Change  
Date                                
2016-01-04  20786   7949903 -0.020  
2016-01-05  20916  14251222  0.006  
2016-01-06  20764  12478969 -0.007  
2016-01-07  20565  12867202 -0.010  
2016-01-08  20711  14064157  0.007  
...           ...       ...    ...  
2017-12-21  28976   9449472 -0.019  
2017-12-22  29115   9360619  0.005  
2017-12-26  28997   8390432 -0.004  
2017-12-27  29338  13964805  0.012  
2017-12-28  29705   8343692  0.013  

[489 rows x 12 columns]

In [60]:
pd.concat([samsung_df, kodex_df[['Close']]], join='inner')

,Close
Date,
2009-01-02,9400
2009-01-05,9520
2009-01-06,9960
2009-01-07,10480
2009-01-08,10160
...,...
2017-12-21,28976
2017-12-22,29115
2017-12-26,28997


In [63]:
samsung_df_copy = samsung_df.copy()
samsung_df_copy.columns = ['1_' + col for col in samsung_df.columns]
pd.concat([samsung_df, samsung_df_copy])

,Open,High,Low,Close,Volume,Change,1_Open,1_High,1_Low,1_Close,1_Volume,1_Change
Date,,,,,,,,,,,,
2009-01-02,9070.000,9400.000,8850.000,9400.000,460533.000,0.042,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,9600.000,9740.000,9470.000,9520.000,609528.000,0.013,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,9640.000,10040.000,9590.000,9960.000,901449.000,0.046,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,10100.000,10480.000,9980.000,10480.000,827163.000,0.052,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,10359.000,10380.000,10060.000,10160.000,613566.000,-0.031,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,NaN,NaN,NaN,NaN,NaN,NaN,51000.000,51060.000,49100.000,49140.000,312486.000,-0.034
2017-12-22,NaN,NaN,NaN,NaN,NaN,NaN,49400.000,49960.000,49240.000,49700.000,223993.000,0.011
2017-12-26,NaN,NaN,NaN,NaN,NaN,NaN,49760.000,50100.000,48200.000,48200.000,320797.000,-0.030


In [66]:
tot_df = pd.concat([samsung_df, kodex_df], keys=['삼성전자','KODEX200'], names=['ticker'])
tot_df = tot_df.reset_index()
tot_df.pivot('Date', 'ticker', 'Close')

ticker,KODEX200,삼성전자
Date,,
2009-01-02,NaN,9400.000
2009-01-05,NaN,9520.000
2009-01-06,NaN,9960.000
2009-01-07,NaN,10480.000
2009-01-08,NaN,10160.000
...,...,...
2017-12-21,28976.000,49140.000
2017-12-22,29115.000,49700.000
2017-12-26,28997.000,48200.000


In [67]:
sample_data = pd.DataFrame(
    {
        "종목명":["삼성", "현대", "하이닉스", "삼성", "현대", "하이닉스"],
        "datetime":["2019-01-01", "2019-01-01", "2019-01-01", "2019-01-02", "2019-01-02", "2019-01-02"],
        "price":[1,2,3, 4,5,6]
    }
)
sample_data.sort_values('종목명')

,종목명,datetime,price
0,삼성,2019-01-01,1
3,삼성,2019-01-02,4
2,하이닉스,2019-01-01,3
5,하이닉스,2019-01-02,6
1,현대,2019-01-01,2
4,현대,2019-01-02,5


In [68]:
sample_data.pivot(index='datetime', columns='종목명', values='price')

종목명,삼성,하이닉스,현대
datetime,,,
2019-01-01,1,3,2
2019-01-02,4,6,5
